# 1. Library Import

In [16]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')


In [17]:
%matplotlib inline

import matplotlib as mpl  # 기본 설정
import matplotlib.pyplot as plt  # 그래프 그리기
import matplotlib.font_manager as fm  # 폰트 관리


!apt-get update -qq         # apt-get 패키지 설치 명령어, -qq : 에러외 메세지 숨기기
!apt-get install fonts-nanum* -qq #나눔글꼴 설치

fe = fm.FontEntry(fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', name='NanumGothic') #파일 저장되어있는 경로와 이름 설정
fm.fontManager.ttflist.insert(0, fe)  # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumGothic'}) #폰트설정



W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


# 2. 데이터 불러오기 및 확인

In [18]:
df_hitter = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/HK/kbo_salary/KBO_Salary/hitter_salary_debut.csv')

In [19]:
df_hitter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 535 entries, 0 to 534
Data columns (total 32 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   순위      535 non-null    int64  
 1   선수명     535 non-null    object 
 2   팀명      535 non-null    object 
 3   AVG     535 non-null    float64
 4   G       535 non-null    int64  
 5   PA      535 non-null    int64  
 6   AB      535 non-null    int64  
 7   R       535 non-null    int64  
 8   H       535 non-null    int64  
 9   2B      535 non-null    int64  
 10  3B      535 non-null    int64  
 11  HR      535 non-null    int64  
 12  TB      535 non-null    int64  
 13  RBI     535 non-null    int64  
 14  SAC     535 non-null    int64  
 15  SF      535 non-null    int64  
 16  BB      535 non-null    int64  
 17  IBB     535 non-null    int64  
 18  HBP     535 non-null    int64  
 19  SO      535 non-null    int64  
 20  GDP     535 non-null    int64  
 21  SLG     535 non-null    float64
 22  OB

In [20]:
df_pf = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/HK/kbo_salary/KBO_Salary/pitcher_meanERA_parkfactor.csv')

df_pf.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 444 entries, 0 to 443
Data columns (total 41 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   순위              444 non-null    int64  
 1   선수명             444 non-null    object 
 2   팀명              444 non-null    object 
 3   ERA             444 non-null    float64
 4   G               444 non-null    int64  
 5   W               444 non-null    int64  
 6   L               444 non-null    int64  
 7   SV              444 non-null    int64  
 8   HLD             444 non-null    int64  
 9   WPCT            444 non-null    float64
 10  IP              444 non-null    float64
 11  H               444 non-null    float64
 12  HR              444 non-null    int64  
 13  BB              444 non-null    int64  
 14  HBP             444 non-null    int64  
 15  SO              444 non-null    int64  
 16  R               444 non-null    int64  
 17  ER              444 non-null    int

In [21]:
df_pf = df_pf[['연도','팀명','park_factor']].drop_duplicates().reset_index()

In [22]:
league = {
    '리그평균출루율' : [0.349,0.346,0.333,0.338],
    '리그평균장타율' : [0.409,0.383,0.379,0.374],
    '연도' : [2020, 2021, 2022, 2023]
}

df_league = pd.DataFrame(league)

In [23]:
def calculate_ops_plus(df_hitter):
    slg = df_hitter['SLG'] / (df_league.loc[df_league['연도'] == df_hitter['연도'],'리그평균장타율'].values[0])

    obp = df_hitter['OBP'] / (df_league.loc[df_league['연도'] == df_hitter['연도'],'리그평균출루율'].values[0])
    pf = df_pf.loc[(df_pf['연도']==df_hitter['연도']) & (df_pf['팀명'] == df_hitter['팀명']),'park_factor'].values[0]

    ops_plus = (slg + obp - 1) * 100 / pf
    return ops_plus

df_hitter['OPS+'] = df_hitter.apply(calculate_ops_plus, axis=1)

In [24]:
df = df_hitter.drop( ['연도','순위','선수명','팀명'] ,axis=1)

df.describe()

,AVG,G,PA,AB,R,H,2B,3B,HR,TB,...,OBP,OPS,MH,RISP,PH-BA,연봉(만원),WAR,후년연봉,연차,OPS+
count,535.000000,535.000000,535.000000,535.000000,535.000000,535.000000,535.000000,535.000000,535.000000,535.000000,...,535.000000,535.000000,535.000000,535.000000,535.000000,535.000000,535.000000,535.000000,535.000000,535.000000
mean,0.242138,84.489720,275.267290,240.028037,34.022430,64.385047,11.022430,1.022430,5.332710,93.450467,...,0.318148,0.663002,16.115888,0.242198,0.214473,25025.663551,1.406336,25481.738318,8.545794,0.083080
std,0.068091,43.120068,199.317472,173.470656,27.689684,51.973392,9.892278,1.653354,7.004918,78.296427,...,0.071765,0.172363,14.922824,0.108728,0.238355,42212.847498,1.920611,37783.821164,5.321258,0.046593
min,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,2700.000000,-2.480000,3000.000000,1.000000,-0.103199
25%,0.215500,47.000000,85.500000,74.000000,10.000000,17.000000,2.000000,0.000000,0.000000,22.000000,...,0.289000,0.590500,3.000000,0.200000,0.000000,3400.000000,-0.040000,4500.000000,4.000000,0.064144
50%,0.251000,92.000000,258.000000,224.000000,28.000000,54.000000,9.000000,0.000000,3.000000,79.000000,...,0.327000,0.684000,12.000000,0.260000,0.182000,7200.000000,0.610000,9000.000000,8.000000,0.088491
75%,0.284000,123.000000,463.500000,406.000000,53.000000,110.500000,18.000000,1.000000,7.000000,153.500000,...,0.362000,0.765500,27.000000,0.305500,0.333000,26000.000000,2.570000,30000.000000,12.000000,0.110818
max,0.750000,144.000000,651.000000,570.000000,115.000000,192.000000,46.000000,11.000000,35.000000,313.000000,...,0.750000,1.750000,59.000000,1.000000,1.000000,270000.000000,7.910000,270000.000000,24.000000,0.376648


# 4. 연봉 구간 다중 분류 모델링

## 4.1 기존 연봉을 연봉 구간으로 나누기

In [25]:
df_hitter['후년연봉'].describe()

,후년연봉
count,535.000000
mean,25481.738318
std,37783.821164
min,3000.000000
25%,4500.000000
50%,9000.000000
75%,30000.000000
max,270000.000000


In [26]:
import pandas as pd

# '후년연봉'을 4분위로 나누어 4개의 집단으로 분류
df_hitter['연봉구간'] = pd.qcut(df_hitter['후년연봉'], q=4, labels=False)

# 각 군집의 경계 확인
cluster_ranges = []
for i in range(df_hitter['연봉구간'].nunique()):
    cluster_min = df_hitter[df_hitter['연봉구간'] == i]['후년연봉'].min()
    cluster_max = df_hitter[df_hitter['연봉구간'] == i]['후년연봉'].max()
    cluster_ranges.append((cluster_min, cluster_max))

# 결과 출력
df_hitter[['후년연봉', '연봉구간']].head(), cluster_ranges

(    후년연봉  연봉구간
 0  50000     3
 1  60000     3
 2  60000     3
 3  40000     3
 4  31000     3,
 [(3000, 4500), (4700, 9000), (9200, 30000), (31000, 270000)])

In [27]:
def assign_salary_range(row):
  if row < 4700:
    return 0
  elif row < 9200:
    return 1
  elif row < 31000:
    return 2
  else:
    return 3

df_hitter['현재연봉구간'] = df_hitter['연봉(만원)'].apply(assign_salary_range)


In [28]:
df_hitter['연봉구간'].value_counts()

,count
연봉구간,
2,148
0,138
1,131
3,118


In [29]:
df_hitter['현재연봉구간'].value_counts()


,count
현재연봉구간,
0,194
2,124
3,111
1,106


# 4. 연봉 구간 다중 분류 모델링

## 4.1 기존 연봉을 연봉 구간으로 나누기

In [30]:
df_hitter['후년연봉'].describe()

,후년연봉
count,535.000000
mean,25481.738318
std,37783.821164
min,3000.000000
25%,4500.000000
50%,9000.000000
75%,30000.000000
max,270000.000000


In [31]:
import pandas as pd

# '후년연봉'을 4분위로 나누어 4개의 집단으로 분류
df_hitter['연봉구간'] = pd.qcut(df_hitter['후년연봉'], q=4, labels=False)

# 각 군집의 경계 확인
cluster_ranges = []
for i in range(df_hitter['연봉구간'].nunique()):
    cluster_min = df_hitter[df_hitter['연봉구간'] == i]['후년연봉'].min()
    cluster_max = df_hitter[df_hitter['연봉구간'] == i]['후년연봉'].max()
    cluster_ranges.append((cluster_min, cluster_max))

# 결과 출력
df_hitter[['후년연봉', '연봉구간']].head(), cluster_ranges

(    후년연봉  연봉구간
 0  50000     3
 1  60000     3
 2  60000     3
 3  40000     3
 4  31000     3,
 [(3000, 4500), (4700, 9000), (9200, 30000), (31000, 270000)])

In [32]:
def assign_salary_range(row):
  if row < 4700:
    return 0
  elif row < 9200:
    return 1
  elif row < 31000:
    return 2
  else:
    return 3

df_hitter['현재연봉구간'] = df_hitter['연봉(만원)'].apply(assign_salary_range)


In [33]:
df_hitter['연봉구간'].value_counts()

,count
연봉구간,
2,148
0,138
1,131
3,118


In [34]:
df_hitter['현재연봉구간'].value_counts()


,count
현재연봉구간,
0,194
2,124
3,111
1,106


## 4.2 전체 컬럼으로 모델링

In [35]:
X = df_hitter.drop(['순위','팀명','선수명','SO','연봉(만원)','연도','후년연봉','연봉구간'],axis=1)
y = df_hitter['연봉구간']

In [36]:
df_hitter.to_csv('hitter_final2.csv',index=False)

In [37]:
X.shape

(535, 27)

In [38]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 535 entries, 0 to 534
Data columns (total 27 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AVG     535 non-null    float64
 1   G       535 non-null    int64  
 2   PA      535 non-null    int64  
 3   AB      535 non-null    int64  
 4   R       535 non-null    int64  
 5   H       535 non-null    int64  
 6   2B      535 non-null    int64  
 7   3B      535 non-null    int64  
 8   HR      535 non-null    int64  
 9   TB      535 non-null    int64  
 10  RBI     535 non-null    int64  
 11  SAC     535 non-null    int64  
 12  SF      535 non-null    int64  
 13  BB      535 non-null    int64  
 14  IBB     535 non-null    int64  
 15  HBP     535 non-null    int64  
 16  GDP     535 non-null    int64  
 17  SLG     535 non-null    float64
 18  OBP     535 non-null    float64
 19  OPS     535 non-null    float64
 20  MH      535 non-null    int64  
 21  RISP    535 non-null    float64
 22  PH

## 4.2.1 train과 test 분리

In [39]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=0
)

In [40]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 374 entries, 252 to 192
Data columns (total 27 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AVG     374 non-null    float64
 1   G       374 non-null    int64  
 2   PA      374 non-null    int64  
 3   AB      374 non-null    int64  
 4   R       374 non-null    int64  
 5   H       374 non-null    int64  
 6   2B      374 non-null    int64  
 7   3B      374 non-null    int64  
 8   HR      374 non-null    int64  
 9   TB      374 non-null    int64  
 10  RBI     374 non-null    int64  
 11  SAC     374 non-null    int64  
 12  SF      374 non-null    int64  
 13  BB      374 non-null    int64  
 14  IBB     374 non-null    int64  
 15  HBP     374 non-null    int64  
 16  GDP     374 non-null    int64  
 17  SLG     374 non-null    float64
 18  OBP     374 non-null    float64
 19  OPS     374 non-null    float64
 20  MH      374 non-null    int64  
 21  RISP    374 non-null    float64
 22  PH-BA

## 4.2.2  모델링

In [41]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()

model.fit(X_train,y_train)

RandomForestClassifier()

In [42]:
from sklearn.metrics import accuracy_score

# test 데이터 예측
y_train_pred = model.predict(X_train)

y_pred = model.predict(X_test)


# 정확도 계산
print(f'Train Accuracy : {model.score(X_train, y_train)}')
print(f'Test Accuracy  : {model.score(X_test, y_test)}')

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy : {accuracy}")

Train Accuracy : 1.0
Test Accuracy  : 0.84472049689441
Accuracy : 0.84472049689441


In [43]:
from sklearn.model_selection import GridSearchCV

# 하이퍼파라미터 튜닝을 위한 파라미터 그리드 설정
params = {
    'n_estimators': [100,300,400],       # 생성할 결정 트리 개수 ( 10, 100, 300 돌렸을 때 300 나와서 크기 늘려서 다시 진행 )
    'max_depth': [6, 8, 10, 12],             # 각 트리의 최대 깊이
    'min_samples_leaf': [8, 12, 18],         # 리프 노드가 되기 위한 최소 샘플 수
    'min_samples_split': [8, 16, 20,24]      # 내부 노드를 분할하기 위한 최소 샘플 수
}

# # RandomForestClassifier 객체 생성 (random_state로 결과 재현 가능, n_jobs로 병렬 처리)
rf_clf = RandomForestClassifier(random_state=0, n_jobs=-1)

# # GridSearchCV 객체 생성 (cv로 교차 검증의 fold 수 설정)
grid_cv = GridSearchCV(rf_clf, param_grid=params, cv=3, n_jobs=-1)

# # 학습 데이터로 GridSearchCV 실행 (최적의 하이퍼파라미터 탐색)
grid_cv.fit(X_train, y_train)

# # 최적의 하이퍼파라미터 출력
print('최적 하이퍼 파라미터: ', grid_cv.best_params_)

# # 최고 예측 정확도 출력
print('최고 예측 정확도: {:.4f}'.format(grid_cv.best_score_))

최적 하이퍼 파라미터:  {'max_depth': 6, 'min_samples_leaf': 8, 'min_samples_split': 20, 'n_estimators': 400}
최고 예측 정확도: 0.7592


In [44]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(max_depth = 6, min_samples_leaf = 8, min_samples_split = 20, n_estimators= 400)

model.fit(X_train,y_train)

RandomForestClassifier(max_depth=6, min_samples_leaf=8, min_samples_split=20,
                       n_estimators=400)

In [45]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

# train 데이터 예측
y_train_pred = model.predict(X_train)

# test 데이터 예측
y_test_pred = model.predict(X_test)

# 정확도 계산
print(f'Train Accuracy : {model.score(X_train, y_train)}')
print(f'Test Accuracy  : {model.score(X_test, y_test)}')

# 훈련 데이터 예측 평가
train_accuracy = accuracy_score(y_train,y_train_pred)
train_f1 = f1_score(y_train,y_train_pred, average = 'weighted')
train_precision = precision_score(y_train,y_train_pred, average = 'weighted')
train_recall = recall_score(y_train,y_train_pred, average = 'weighted')

# 테스트 데이터 예측 평가
test_accuracy = accuracy_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred, average = 'weighted')
test_precision = precision_score(y_test, y_test_pred, average = 'weighted')
test_recall = recall_score(y_test, y_test_pred, average = 'weighted')



print(f"Train 정확도 : {train_accuracy}")
print(f"Train 정밀도 : {train_precision}")
print(f"Train 재현율 : {train_recall}")
print(f'Train F1 score : {train_f1}')

print(f"Test 정확도 : {test_accuracy}")
print(f"Test 정밀도 : {test_precision}")
print(f"Test 재현율 : {test_recall}")
print(f'Test F1 score : {test_f1}')


Train Accuracy : 0.8850267379679144
Test Accuracy  : 0.8074534161490683
Train 정확도 : 0.8850267379679144
Train 정밀도 : 0.8851387584345763
Train 재현율 : 0.8850267379679144
Train F1 score : 0.885044091871408
Test 정확도 : 0.8074534161490683
Test 정밀도 : 0.8259686483288967
Test 재현율 : 0.8074534161490683
Test F1 score : 0.8112970142445063


**과적합**

## 4.2

In [46]:
from sklearn.model_selection import train_test_split


data = df_hitter[['H','BB','HR','RBI','OPS+','WAR','MH','TB','연차','현재연봉구간']]
target = df_hitter['연봉구간']

X_train, X_test, y_train, y_test = train_test_split(
    data,
    target,
    test_size=0.3,
    random_state=0
)

In [47]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(max_depth = 6, min_samples_leaf = 12, min_samples_split = 8, n_estimators= 300)

model.fit(X_train,y_train)

# train 데이터 예측
y_train_pred = model.predict(X_train)

# test 데이터 예측
y_test_pred = model.predict(X_test)

# 정확도 계산
print(f'Train Accuracy : {model.score(X_train, y_train)}')
print(f'Test Accuracy  : {model.score(X_test, y_test)}')

# 훈련 데이터 예측 평가
train_accuracy = accuracy_score(y_train,y_train_pred)
train_f1 = f1_score(y_train,y_train_pred, average = 'weighted')
train_precision = precision_score(y_train,y_train_pred, average = 'weighted')
train_recall = recall_score(y_train,y_train_pred, average = 'weighted')

# 테스트 데이터 예측 평가
test_accuracy = accuracy_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred, average = 'weighted')
test_precision = precision_score(y_test, y_test_pred, average = 'weighted')
test_recall = recall_score(y_test, y_test_pred, average = 'weighted')



print(f"Train 정확도 : {train_accuracy}")
print(f"Train 정밀도 : {train_precision}")
print(f"Train 재현율 : {train_recall}")
print(f'Train F1 score : {train_f1}')

print(f"Test 정확도 : {test_accuracy}")
print(f"Test 정밀도 : {test_precision}")
print(f"Test 재현율 : {test_recall}")
print(f'Test F1 score : {test_f1}')


Train Accuracy : 0.8556149732620321
Test Accuracy  : 0.8198757763975155
Train 정확도 : 0.8556149732620321
Train 정밀도 : 0.8548340711193941
Train 재현율 : 0.8556149732620321
Train F1 score : 0.855090837950862
Test 정확도 : 0.8198757763975155
Test 정밀도 : 0.8318035426731081
Test 재현율 : 0.8198757763975155
Test F1 score : 0.8230612380411332


# 4.3

In [49]:
from sklearn.model_selection import train_test_split


data = df_hitter[['TB', 'H', '연차', 'AB', 'RBI', 'MH', 'R', 'BB', '2B', 'IBB', 'WAR',
       'SLG', 'OPS+', 'AVG', 'HR','현재연봉구간']]
target = df_hitter['연봉구간']

X_train, X_test, y_train, y_test = train_test_split(
    data,
    target,
    test_size=0.3,
    random_state=0
)

In [50]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(max_depth = 6, min_samples_leaf = 12, min_samples_split = 8, n_estimators= 300)

model.fit(X_train,y_train)

# train 데이터 예측
y_train_pred = model.predict(X_train)

# test 데이터 예측
y_test_pred = model.predict(X_test)

# 정확도 계산
print(f'Train Accuracy : {model.score(X_train, y_train)}')
print(f'Test Accuracy  : {model.score(X_test, y_test)}')

# 훈련 데이터 예측 평가
train_accuracy = accuracy_score(y_train,y_train_pred)
train_f1 = f1_score(y_train,y_train_pred, average = 'weighted')
train_precision = precision_score(y_train,y_train_pred, average = 'weighted')
train_recall = recall_score(y_train,y_train_pred, average = 'weighted')

# 테스트 데이터 예측 평가
test_accuracy = accuracy_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred, average = 'weighted')
test_precision = precision_score(y_test, y_test_pred, average = 'weighted')
test_recall = recall_score(y_test, y_test_pred, average = 'weighted')



print(f"Train 정확도 : {train_accuracy}")
print(f"Train 정밀도 : {train_precision}")
print(f"Train 재현율 : {train_recall}")
print(f'Train F1 score : {train_f1}')

print(f"Test 정확도 : {test_accuracy}")
print(f"Test 정밀도 : {test_precision}")
print(f"Test 재현율 : {test_recall}")
print(f'Test F1 score : {test_f1}')


Train Accuracy : 0.8609625668449198
Test Accuracy  : 0.8260869565217391
Train 정확도 : 0.8609625668449198
Train 정밀도 : 0.860762256571213
Train 재현율 : 0.8609625668449198
Train F1 score : 0.8606465726786582
Test 정확도 : 0.8260869565217391
Test 정밀도 : 0.8387787650945544
Test 재현율 : 0.8260869565217391
Test F1 score : 0.8294350477274128


# 4.4

In [69]:
from sklearn.model_selection import train_test_split


data = df_hitter[['TB', 'H', '연차', 'AB', 'RBI', 'MH', 'R', 'BB', '2B', 'IBB', 'WAR','현재연봉구간']]
target = df_hitter['연봉구간']

X_train, X_test, y_train, y_test = train_test_split(
    data,
    target,
    test_size=0.3,
    random_state=0
)

In [62]:
from sklearn.model_selection import GridSearchCV

# 하이퍼파라미터 튜닝을 위한 파라미터 그리드 설정
params = {
    'n_estimators': [100,300,400],       # 생성할 결정 트리 개수 ( 10, 100, 300 돌렸을 때 300 나와서 크기 늘려서 다시 진행 )
    'max_depth': [6, 8, 10, 12],             # 각 트리의 최대 깊이
    'min_samples_leaf': [8, 12, 18],         # 리프 노드가 되기 위한 최소 샘플 수
    'min_samples_split': [8, 16, 20,24]      # 내부 노드를 분할하기 위한 최소 샘플 수
}

# # RandomForestClassifier 객체 생성 (random_state로 결과 재현 가능, n_jobs로 병렬 처리)
rf_clf = RandomForestClassifier(random_state=0, n_jobs=-1)

# # GridSearchCV 객체 생성 (cv로 교차 검증의 fold 수 설정)
grid_cv = GridSearchCV(rf_clf, param_grid=params, cv=3, n_jobs=-1)

# # 학습 데이터로 GridSearchCV 실행 (최적의 하이퍼파라미터 탐색)
grid_cv.fit(X_train, y_train)

# # 최적의 하이퍼파라미터 출력
print('최적 하이퍼 파라미터: ', grid_cv.best_params_)

# # 최고 예측 정확도 출력
print('최고 예측 정확도: {:.4f}'.format(grid_cv.best_score_))


최적 하이퍼 파라미터:  {'max_depth': 6, 'min_samples_leaf': 8, 'min_samples_split': 8, 'n_estimators': 100}
최고 예측 정확도: 0.7940


In [70]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(max_depth = 6, min_samples_leaf = 12, min_samples_split = 8, n_estimators= 300)

model.fit(X_train,y_train)

# train 데이터 예측
y_train_pred = model.predict(X_train)

# test 데이터 예측
y_test_pred = model.predict(X_test)

# 정확도 계산
print(f'Train Accuracy : {model.score(X_train, y_train)}')
print(f'Test Accuracy  : {model.score(X_test, y_test)}')

# 훈련 데이터 예측 평가
train_accuracy = accuracy_score(y_train,y_train_pred)
train_f1 = f1_score(y_train,y_train_pred, average = 'weighted')
train_precision = precision_score(y_train,y_train_pred, average = 'weighted')
train_recall = recall_score(y_train,y_train_pred, average = 'weighted')

# 테스트 데이터 예측 평가
test_accuracy = accuracy_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred, average = 'weighted')
test_precision = precision_score(y_test, y_test_pred, average = 'weighted')
test_recall = recall_score(y_test, y_test_pred, average = 'weighted')



print(f"Train 정확도 : {train_accuracy}")
print(f"Train 정밀도 : {train_precision}")
print(f"Train 재현율 : {train_recall}")
print(f'Train F1 score : {train_f1}')

print(f"Test 정확도 : {test_accuracy}")
print(f"Test 정밀도 : {test_precision}")
print(f"Test 재현율 : {test_recall}")
print(f'Test F1 score : {test_f1}')


Train Accuracy : 0.8556149732620321
Test Accuracy  : 0.8198757763975155
Train 정확도 : 0.8556149732620321
Train 정밀도 : 0.8546591160142909
Train 재현율 : 0.8556149732620321
Train F1 score : 0.8547838272124035
Test 정확도 : 0.8198757763975155
Test 정밀도 : 0.8298658596783809
Test 재현율 : 0.8198757763975155
Test F1 score : 0.8229448466307648


In [68]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       1.00      0.92      0.96        38
           1       0.79      0.79      0.79        34
           2       0.70      0.80      0.75        46
           3       0.87      0.79      0.83        43

    accuracy                           0.83       161
   macro avg       0.84      0.83      0.83       161
weighted avg       0.84      0.83      0.83       161

